#  Pandas - Remodelagem e tabelas dinâmicas (pivot_table,crosstab)

In [99]:
import numpy as np
import pandas as pd
import os

diratual = os.getcwd()

diretorio = os.path.join(diratual) 

In [100]:
dfcovid = pd.read_csv(os.path.join(diretorio, 'HIST_PAINEL_COVIDBR_02nov2024.csv') , sep=';')
dfcovid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0.0,0,0,0,0.0,0.0,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1.0,1,0,0,1.0,0.0,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1.0,0,0,0,1.0,0.0,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1.0,0,0,0,0.0,1.0,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2.0,1,0,0,1.0,1.0,NaN


In [101]:
filtro = (dfcovid['estado'].isna() == False ) & ( dfcovid['codmun'].isna()  )
dfcovid = dfcovid[filtro][['regiao', 'estado', 'data' , 'populacaoTCU2019',
       'casosNovos', 'obitosNovos']]

dfcovid['mes'] = dfcovid['data'].map(lambda x : x[:7])


print(dfcovid.shape)
dfcovid.head()

(46251, 7)


,regiao,estado,data,populacaoTCU2019,casosNovos,obitosNovos,mes
158,Norte,RO,2020-02-25,1777225.0,0,0,2020-02
159,Norte,RO,2020-02-26,1777225.0,0,0,2020-02
160,Norte,RO,2020-02-27,1777225.0,0,0,2020-02
161,Norte,RO,2020-02-28,1777225.0,0,0,2020-02
162,Norte,RO,2020-02-29,1777225.0,0,0,2020-02


In [102]:
dfregiaouf = dfcovid[['regiao', 'estado', 'mes',  'casosNovos', 'obitosNovos', 'populacaoTCU2019', 'data']]\
  .groupby(['regiao', 'estado', 'mes', 'data' ])\
  .agg({'casosNovos':'sum', 'obitosNovos':'sum', 'populacaoTCU2019':'max'})\
  .reset_index()
dfregiaouf.head()

,regiao,estado,mes,data,casosNovos,obitosNovos,populacaoTCU2019
0,Centro-Oeste,DF,2020-02,2020-02-25,0,0,3015268.0
1,Centro-Oeste,DF,2020-02,2020-02-26,0,0,3015268.0
2,Centro-Oeste,DF,2020-02,2020-02-27,0,0,3015268.0
3,Centro-Oeste,DF,2020-02,2020-02-28,0,0,3015268.0
4,Centro-Oeste,DF,2020-02,2020-02-29,0,0,3015268.0


In [103]:
dfregiaouf.columns

Index(['regiao', 'estado', 'mes', 'data', 'casosNovos', 'obitosNovos',
       'populacaoTCU2019'],
      dtype='object')

# Remodelagem e tabelas dinâmicas 

## Tabelas dinâmicas 
O Pandas possui dois métodos para gerar tabelas dinâmicas:
- pivot() fornece pivotamento de propósito geral com vários tipos de dados (strings, numéricos, etc.) 
- pivot_table() fornece pivotamento com agregação de dados numéricos.

## pivot_table() 
A função pivot_table()pode ser usada para criar tabelas dinâmicas no estilo planilha. 
- Utiliza os seguintes argumentos:

 - data: um objeto DataFrame.

 - values: uma coluna ou uma lista de colunas para agregar.

 - index: uma coluna, Grouper, array que tem o mesmo comprimento dos dados, ou uma lista deles. Chaves para agrupar no índice da tabela dinâmica. Se um array é passado, ele está sendo usado da mesma maneira que os valores das colunas.

 - columns: uma coluna, Grouper, array que tem o mesmo comprimento dos dados, ou uma lista deles. Chaves para agrupar na coluna da tabela dinâmica. Se um array é passado, ele está sendo usado da mesma maneira que os valores das colunas.

 - aggfunc: função a ser usada para agregação, assumindo como **padrão numpy.mean**

In [104]:
dfresumo = pd.pivot_table(dfregiaouf, values=['casosNovos', 'obitosNovos'], 
               index=['mes'], columns=['regiao', 'estado'])
dfresumo.head(5)

casosNovos                                                  \
regiao  Centro-Oeste                                        Nordeste   
estado            DF          GO          MS          MT          AL   
mes                                                                    
2020-02     0.000000    0.000000    0.000000    0.000000    0.000000   
2020-03    10.709677    2.096774    1.548387    0.806452    0.580645   
2020-04    34.133333   23.866667    6.900000    9.066667   34.200000   
2020-05   271.741935   94.225806   39.806452   68.774194  298.193548   
2020-06  1314.600000  689.300000  215.866667  440.233333  855.800000   

                                                                         ...  \
regiao                                                                   ...   
estado            BA           CE           MA           PB          PE  ...   
mes                                                                      ...   
2020-02     0.000000     0.000000     0.000000     0.000000    0.000000  ...   
2020-03     6.870968    12.580645     1.000000     0.548387    2.806452  ...   
2020-04    87.933333   240.533333   105.300000    26.566667  226.300000  ...   
2020-05   501.322581  1318.806452  1014.483871   398.322581  889.483871  ...   
2020-06  1830.500000  2007.000000  1527.066667  1126.500000  813.600000  ...   

        obitosNovos                                                        \
regiao        Norte                        Sudeste                          
estado           RO        RR        TO         ES         MG          RJ   
mes                                                                         
2020-02    0.000000  0.000000  0.000000   0.000000   0.000000    0.000000   
2020-03    0.032258  0.000000  0.000000   0.000000   0.064516    0.741935   
2020-04    0.500000  0.233333  0.100000   2.766667   2.666667   27.700000   
2020-05    4.516129  3.516129  2.258065  16.806452   6.096774  144.838710   
2020-06   12.066667  5.566667  4.233333  34.800000  23.133333  157.866667   

                                                     
regiao                     Sul                       
estado           SP         PR         RS        SC  
mes                                                  
2020-02    0.000000   0.000000   0.000000  0.000000  
2020-03    4.387097   0.096774   0.129032  0.064516  
2020-04   74.633333   2.666667   1.566667  1.466667  
2020-05  169.032258   3.193548   5.580645  3.129032  
2020-06  238.266667  15.566667  13.000000  6.600000  

[5 rows x 54 columns]

In [105]:
dfresumo = pd.pivot_table(dfregiaouf, values=['casosNovos', 'obitosNovos'], 
               index=['regiao', 'estado'], columns=['mes'])
dfresumo.head(5)

casosNovos                                                 \
mes                    2020-02    2020-03    2020-04     2020-05      2020-06   
regiao       estado                                                             
Centro-Oeste DF            0.0  10.709677  34.133333  271.741935  1314.600000   
             GO            0.0   2.096774  23.866667   94.225806   689.300000   
             MS            0.0   1.548387   6.900000   39.806452   215.866667   
             MT            0.0   0.806452   9.066667   68.774194   440.233333   
Nordeste     AL            0.0   0.580645  34.200000  298.193548   855.800000   

                                                                         \
mes                      2020-07      2020-08      2020-09      2020-10   
regiao       estado                                                       
Centro-Oeste DF      1841.096774  1798.387097  1006.766667   677.419355   
             GO      1405.161290  2082.741935  2563.500000  1485.032258   
             MS       547.451613   774.225806   692.300000   407.677419   
             MT      1168.677419  1273.580645  1069.000000   628.580645   
Nordeste     AL       766.548387   615.451613   269.166667   126.129032   

                                 ... obitosNovos                      \
mes                     2020-11  ...     2024-02   2024-03   2024-04   
regiao       estado              ...                                   
Centro-Oeste DF      530.033333  ...    0.689655  0.741935  0.300000   
             GO      798.866667  ...    0.827586  1.838710  0.233333   
             MS      557.233333  ...    0.413793  1.032258  0.166667   
             MT      507.700000  ...    1.103448  0.612903  0.200000   
Nordeste     AL      139.833333  ...    0.275862  0.193548  0.133333   

                                                                       \
mes                   2024-05   2024-06   2024-07   2024-08   2024-09   
regiao       estado                                                     
Centro-Oeste DF      0.096774  0.066667  0.096774  0.064516  0.166667   
             GO      0.322581  0.300000  0.193548  0.387097  1.266667   
             MS      0.161290  0.066667  0.064516  0.516129  0.900000   
             MT      0.064516  0.066667  0.032258  0.290323  0.000000   
Nordeste     AL      0.000000  0.000000  0.096774  0.193548  0.033333   

                                       
mes                   2024-10 2024-11  
regiao       estado                    
Centro-Oeste DF      0.193548     0.0  
             GO      0.870968     0.0  
             MS      0.322581     0.5  
             MT      0.387097     0.0  
Nordeste     AL      0.032258     0.0  

[5 rows x 116 columns]

In [106]:
dfresumo = pd.pivot_table(dfregiaouf, values=['casosNovos', 'obitosNovos'], 
               index=['mes'], columns=['regiao', 'estado'], 
               aggfunc="sum")
dfresumo.head(5)

casosNovos                                                           \
regiao  Centro-Oeste                     Nordeste                               
estado            DF     GO    MS     MT       AL     BA     CE     MA     PB   
mes                                                                             
2020-02            0      0     0      0        0      0      0      0      0   
2020-03          332     65    48     25       18    213    390     31     17   
2020-04         1024    716   207    272     1026   2638   7216   3159    797   
2020-05         8424   2921  1234   2132     9244  15541  40883  31449  12348   
2020-06        39438  20679  6476  13207    25674  54915  60210  45812  33795   

                ... obitosNovos                                               \
regiao          ...       Norte           Sudeste                   Sul        
estado      PE  ...          RO   RR   TO      ES   MG    RJ    SP   PR   RS   
mes             ...                                                            
2020-02      0  ...           0    0    0       0    0     0     0    0    0   
2020-03     87  ...           1    0    0       0    2    23   136    3    4   
2020-04   6789  ...          15    7    3      83   80   831  2239   80   47   
2020-05  27574  ...         140  109   70     521  189  4490  5240   99  173   
2020-06  24408  ...         362  167  127    1044  694  4736  7148  467  390   

              
regiao        
estado    SC  
mes           
2020-02    0  
2020-03    2  
2020-04   44  
2020-05   97  
2020-06  198  

[5 rows x 54 columns]

In [107]:
dfresumo = pd.pivot_table(dfregiaouf, values=['casosNovos', 'obitosNovos'], 
               index=['regiao', 'estado'], columns=['mes'],
                aggfunc="sum")
dfresumo.head(5)

casosNovos                                          \
mes                    2020-02 2020-03 2020-04 2020-05 2020-06 2020-07   
regiao       estado                                                      
Centro-Oeste DF              0     332    1024    8424   39438   57074   
             GO              0      65     716    2921   20679   43560   
             MS              0      48     207    1234    6476   16971   
             MT              0      25     272    2132   13207   36229   
Nordeste     AL              0      18    1026    9244   25674   23763   

                                                     ... obitosNovos          \
mes                 2020-08 2020-09 2020-10 2020-11  ...     2024-02 2024-03   
regiao       estado                                  ...                       
Centro-Oeste DF       55750   30203   21000   15901  ...          20      23   
             GO       64565   76905   46036   23966  ...          24      57   
             MS       24001   20769   12638   16717  ...          12      32   
             MT       39481   32070   19486   15231  ...          32      19   
Nordeste     AL       19079    8075    3910    4195  ...           8       6   

                                                                             \
mes                 2024-04 2024-05 2024-06 2024-07 2024-08 2024-09 2024-10   
regiao       estado                                                           
Centro-Oeste DF           9       3       2       3       2       5       6   
             GO           7      10       9       6      12      38      27   
             MS           5       5       2       2      16      27      10   
             MT           6       2       2       1       9       0      12   
Nordeste     AL           4       0       0       3       6       1       1   

                             
mes                 2024-11  
regiao       estado          
Centro-Oeste DF           0  
             GO           0  
             MS           1  
             MT           0  
Nordeste     AL           0  

[5 rows x 116 columns]

### O pivot_tabletambém está disponível como um método de instância no DataFrame, ie DataFrame.pivot_table().

In [108]:
dfresumo = dfregiaouf.pivot_table(values=['casosNovos', 'obitosNovos'], 
               index=['regiao', 'estado'], columns=['mes'],
                aggfunc="sum")
dfresumo.head(5)

casosNovos                                          \
mes                    2020-02 2020-03 2020-04 2020-05 2020-06 2020-07   
regiao       estado                                                      
Centro-Oeste DF              0     332    1024    8424   39438   57074   
             GO              0      65     716    2921   20679   43560   
             MS              0      48     207    1234    6476   16971   
             MT              0      25     272    2132   13207   36229   
Nordeste     AL              0      18    1026    9244   25674   23763   

                                                     ... obitosNovos          \
mes                 2020-08 2020-09 2020-10 2020-11  ...     2024-02 2024-03   
regiao       estado                                  ...                       
Centro-Oeste DF       55750   30203   21000   15901  ...          20      23   
             GO       64565   76905   46036   23966  ...          24      57   
             MS       24001   20769   12638   16717  ...          12      32   
             MT       39481   32070   19486   15231  ...          32      19   
Nordeste     AL       19079    8075    3910    4195  ...           8       6   

                                                                             \
mes                 2024-04 2024-05 2024-06 2024-07 2024-08 2024-09 2024-10   
regiao       estado                                                           
Centro-Oeste DF           9       3       2       3       2       5       6   
             GO           7      10       9       6      12      38      27   
             MS           5       5       2       2      16      27      10   
             MT           6       2       2       1       9       0      12   
Nordeste     AL           4       0       0       3       6       1       1   

                             
mes                 2024-11  
regiao       estado          
Centro-Oeste DF           0  
             GO           0  
             MS           1  
             MT           0  
Nordeste     AL           0  

[5 rows x 116 columns]

## Adicionando margens (Totais)
Se você passar margins=True para pivot_table, Allcolunas e linhas especiais serão adicionadas com agregações parciais de grupos nas categorias nas linhas e colunas:

In [109]:
dfresumo = dfregiaouf.pivot_table(values=['obitosNovos'], 
               index=['regiao', 'estado'], columns=['mes'],
                aggfunc="sum",
                margins=True, margins_name = 'Total')
dfresumo.head(28)

obitosNovos                                          \
mes                     2020-02 2020-03 2020-04 2020-05 2020-06 2020-07   
regiao       estado                                                       
Centro-Oeste DF               0       3      27     140     417     882   
             GO               0       1      28      95     350    1182   
             MS               0       1       8      11      56     300   
             MT               0       0      11      50     532    1226   
Nordeste     AL               0       1      46     396     609     515   
             BA               0       2     102     563    1186    1610   
             CE               0       7     475    2528    3136    1522   
             MA               0       1     183     771    1093     965   
             PB               0       0      62     298     617     834   
             PE               0       6     559    2242    2022    1728   
             PI               0       4      20     137     502     666   
             RN               0       1      55     249     729     743   
             SE               0       0      12     146     518     758   
Norte        AC               0       0      19     129     217     166   
             AM               0       3     422    1627     771     445   
             AP               0       0      34     188     195     148   
             PA               0       0     208    2715    1997     808   
             RO               0       1      15     140     362     354   
             RR               0       0       7     109     167     222   
             TO               0       0       3      70     127     181   
Sudeste      ES               0       0      83     521    1044     897   
             MG               0       2      80     189     694    1804   
             RJ               0      23     831    4490    4736    3397   
             SP               0     136    2239    5240    7148    8234   
Sul          PR               0       3      80      99     467    1271   
             RS               0       4      47     173     390    1262   
             SC               0       2      44      97     198     761   
Total                         0     201    5700   23413   30280   32881   

                                                     ...                  \
mes                 2020-08 2020-09 2020-10 2020-11  ... 2024-03 2024-04   
regiao       estado                                  ...                   
Centro-Oeste DF        1052     734     428     247  ...      23       9   
             GO        1438    1578    1081     598  ...      57       7   
             MS         486     441     289     177  ...      32       5   
             MT         932     655     392     287  ...      19       6   
Nordeste     AL         320     185     166     101  ...       6       4   
             BA        1934    1347     878     646  ...      31      12   
             CE         741     585     359     259  ...       0       0   
             MA         433     310     297     243  ...       3       1   
             PB         639     374     277     194  ...       6       1   
             PE        1036     658     376     410  ...      21       8   
             PI         496     302     273     224  ...       1       1   
             RN         479     137     184     111  ...      -5       2   
             SE         423     179     170     102  ...      10       1   
Norte        AC          81      47      34      30  ...       1       0   
             AM         381     507     387     351  ...       3       2   
             AP          96      48      39      59  ...       1       0   
             PA         418     427     170     172  ...      23       4   
             RO         276     209     100     104  ...       9       1   
             RR          82      61      44      35  ...       0       1   
             TO       

## Tabulações cruzadas 


- crosstab() é usado para calcular uma tabulação cruzada de dois (ou mais) fatores. 
- Por padrão, crosstabcalcula uma tabela de frequência dos fatores
- Se pode fazer outros calculos se uma matriz de valores e uma função de agregação for passadas.

Esse método aceita vários parâmetros

- index: tipo array, valores para agrupar nas linhas.

- columns: tipo array, valores para agrupar nas colunas.

- values: tipo array, opcional, array de valores para agregar de acordo com os fatores.

- aggfunc: função, opcional, se nenhum array de valores for passado, calcula uma tabela de frequência.

- rownames: sequência, padrão None, deve corresponder ao número de arrays de linha passadas.

- colnames: sequência, padrão None, se passado, deve corresponder ao número de arrays de coluna passadas.

- margins: booleano, padrão False, Adicionar margens de linha / coluna (subtotais)

- normalize: booleano, {'all', 'index', 'columns'} ou {0,1}, padrão False. Normalize dividindo todos os valores pela soma dos valores.

In [110]:
dfresumo = pd.crosstab(dfregiaouf['mes'], [dfregiaouf['regiao'], dfregiaouf['estado']], 
                       rownames=['Meses'], colnames=['Região', 'Estados'])
dfresumo.head(5)

Região  Centro-Oeste             Nordeste                      ... Norte      \
Estados           DF  GO  MS  MT       AL  BA  CE  MA  PB  PE  ...    RO  RR   
Meses                                                          ...             
2020-02            5   5   5   5        5   5   5   5   5   5  ...     5   5   
2020-03           31  31  31  31       31  31  31  31  31  31  ...    31  31   
2020-04           30  30  30  30       30  30  30  30  30  30  ...    30  30   
2020-05           31  31  31  31       31  31  31  31  31  31  ...    31  31   
2020-06           30  30  30  30       30  30  30  30  30  30  ...    30  30   

Região      Sudeste             Sul          
Estados  TO      ES  MG  RJ  SP  PR  RS  SC  
Meses                                        
2020-02   5       5   5   5   5   5   5   5  
2020-03  31      31  31  31  31  31  31  31  
2020-04  30      30  30  30  30  30  30  30  
2020-05  31      31  31  31  31  31  31  31  
2020-06  30      30  30  30  30  30  30  30  

[5 rows x 27 columns]

In [111]:
dfresumo = pd.crosstab(dfregiaouf['mes'], [dfregiaouf['regiao'], dfregiaouf['estado']], 
                       rownames=['Meses'], colnames=['Região', 'Estados'],
                       values=dfregiaouf['obitosNovos'], aggfunc="sum")
dfresumo.head(5)

Região  Centro-Oeste               Nordeste                               ...  \
Estados           DF   GO  MS   MT       AL    BA    CE    MA   PB    PE  ...   
Meses                                                                     ...   
2020-02            0    0   0    0        0     0     0     0    0     0  ...   
2020-03            3    1   1    0        1     2     7     1    0     6  ...   
2020-04           27   28   8   11       46   102   475   183   62   559  ...   
2020-05          140   95  11   50      396   563  2528   771  298  2242  ...   
2020-06          417  350  56  532      609  1186  3136  1093  617  2022  ...   

Região  Norte           Sudeste                   Sul            
Estados    RO   RR   TO      ES   MG    RJ    SP   PR   RS   SC  
Meses                                                            
2020-02     0    0    0       0    0     0     0    0    0    0  
2020-03     1    0    0       0    2    23   136    3    4    2  
2020-04    15    7    3      83   80   831  2239   80   47   44  
2020-05   140  109   70     521  189  4490  5240   99  173   97  
2020-06   362  167  127    1044  694  4736  7148  467  390  198  

[5 rows x 27 columns]

Se crosstab  receber apenas duas Séries, resultará uma tabela de frequência.

In [112]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'])
dfresumo.head(5)

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,20,45,35,20,15
2020-03,124,279,217,124,93
2020-04,120,270,210,120,90
2020-05,124,279,217,124,93
2020-06,120,270,210,120,90


In [113]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'], values=dfregiaouf['obitosNovos'], aggfunc="sum", margins=True)
dfresumo.head(5)

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul,All
mes,,,,,,
2020-02,0,0,0,0,0,0
2020-03,5,22,4,161,9,201
2020-04,74,1514,708,3233,171,5700
2020-05,296,7330,4978,10440,369,23413
2020-06,1355,10412,3836,13622,1055,30280


### Normalização dos dados
As tabelas de frequência também podem ser normalizadas para mostrar porcentagens em vez de contagens usando o argumento normalize

In [114]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize=True,
                      values=dfregiaouf['obitosNovos'], aggfunc="sum")
print(dfresumo.sum())
print(dfresumo.sum().sum())
dfresumo.head(5)

regiao
Centro-Oeste    0.094180
Nordeste        0.191796
Norte           0.072981
Sudeste         0.482492
Sul             0.158551
dtype: float64
1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.000007,0.000031,0.000006,0.000225,0.000013
2020-04,0.000104,0.002120,0.000991,0.004528,0.000239
2020-05,0.000415,0.010265,0.006971,0.014620,0.000517
2020-06,0.001898,0.014581,0.005372,0.019076,0.001477


normalize também pode normalizar valores em cada linha ou em cada coluna:

In [115]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='columns',
                      values=dfregiaouf['obitosNovos'], aggfunc="sum")
print(dfresumo.sum())
print(dfresumo.sum().sum())
dfresumo.head(5)

regiao
Centro-Oeste    1.0
Nordeste        1.0
Norte           1.0
Sudeste         1.0
Sul             1.0
dtype: float64
5.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.000074,0.000161,0.000077,0.000467,0.000079
2020-04,0.001100,0.011055,0.013586,0.009384,0.001510
2020-05,0.004401,0.053520,0.095521,0.030302,0.003259
2020-06,0.020148,0.076024,0.073608,0.039537,0.009318


In [116]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='index',
                      values=dfregiaouf['obitosNovos'], aggfunc="sum")
print(dfresumo.sum(1))
print(dfresumo.sum(1).sum())
dfresumo.head(5)

mes
2020-02    0.0
2020-03    1.0
2020-04    1.0
2020-05    1.0
2020-06    1.0
2020-07    1.0
2020-08    1.0
2020-09    1.0
2020-10    1.0
2020-11    1.0
2020-12    1.0
2021-01    1.0
2021-02    1.0
2021-03    1.0
2021-04    1.0
2021-05    1.0
2021-06    1.0
2021-07    1.0
2021-08    1.0
2021-09    1.0
2021-10    1.0
2021-11    1.0
2021-12    1.0
2022-01    1.0
2022-02    1.0
2022-03    1.0
2022-04    1.0
2022-05    1.0
2022-06    1.0
2022-07    1.0
2022-08    1.0
2022-09    1.0
2022-10    1.0
2022-11    1.0
2022-12    1.0
2023-01    1.0
2023-02    1.0
2023-03    1.0
2023-04    1.0
2023-05    1.0
2023-06    1.0
2023-07    1.0
2023-08    1.0
2023-09    1.0
2023-10    1.0
2023-11    1.0
2023-12    1.0
2024-01    1.0
2024-02    1.0
2024-03    1.0
2024-04    1.0
2024-05    1.0
2024-06    1.0
2024-07    1.0
2024-08    1.0
2024-09    1.0
2024-10    1.0
2024-11    1.0
dtype: float64
57.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.024876,0.109453,0.019900,0.800995,0.044776
2020-04,0.012982,0.265614,0.124211,0.567193,0.030000
2020-05,0.012643,0.313074,0.212617,0.445906,0.015760
2020-06,0.044749,0.343857,0.126684,0.449868,0.034841


In [117]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='all',
                      values=dfregiaouf['obitosNovos'], aggfunc="sum")
print(dfresumo.sum())
print(dfresumo.sum().sum())
print(dfresumo.sum(1))
print(dfresumo.sum(1).sum())
dfresumo.head(5)

regiao
Centro-Oeste    0.094180
Nordeste        0.191796
Norte           0.072981
Sudeste         0.482492
Sul             0.158551
dtype: float64
1.0
mes
2020-02    0.000000
2020-03    0.000281
2020-04    0.007982
2020-05    0.032788
2020-06    0.042404
2020-07    0.046047
2020-08    0.040480
2020-09    0.031609
2020-10    0.022311
2020-11    0.018536
2020-12    0.030569
2021-01    0.041389
2021-02    0.042626
2021-03    0.093229
2021-04    0.115206
2021-05    0.082638
2021-06    0.077408
2021-07    0.053641
2021-08    0.033670
2021-09    0.022877
2021-10    0.015510
2021-11    0.009603
2021-12    0.006127
2022-01    0.011318
2022-02    0.031082
2022-03    0.014598
2022-04    0.005238
2022-05    0.004452
2022-06    0.006638
2022-07    0.009940
2022-08    0.007634
2022-09    0.002900
2022-10    0.002970
2022-11    0.002302
2022-12    0.005674
2023-01    0.004511
2023-02    0.002819
2023-03    0.001613
2023-04    0.002232
2023-05    0.001504
2023-06    0.001480
2023-07    0.001162
2023-

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.000007,0.000031,0.000006,0.000225,0.000013
2020-04,0.000104,0.002120,0.000991,0.004528,0.000239
2020-05,0.000415,0.010265,0.006971,0.014620,0.000517
2020-06,0.001898,0.014581,0.005372,0.019076,0.001477


### Adicionando margens total
Finalmente, também se pode adicionar margens ou normalizar esta saída.

In [118]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='all',
                      values=dfregiaouf['obitosNovos'], aggfunc="sum",
                      margins=True, margins_name = 'Total')
dfresumo.tail(5)

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul,Total
mes,,,,,,
2024-08,0.000055,0.000043,0.000053,0.000325,0.000008,0.000485
2024-09,0.000098,0.000022,0.000015,0.000399,0.000130,0.000665
2024-10,0.000077,0.000035,0.000021,0.000340,0.000053,0.000527
2024-11,0.000001,0.000001,0.000004,0.000113,0.000036,0.000157
Total,0.094180,0.191796,0.072981,0.482492,0.158551,1.000000


In [119]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='columns',
                      values=dfregiaouf['obitosNovos'], aggfunc="sum",
                      margins=True, margins_name = 'Total')
print(dfresumo.Total.sum())
dfresumo.tail(5)

1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul,Total
mes,,,,,,
2024-07,0.000178,0.000110,0.000134,0.000551,0.000071,0.000325
2024-08,0.000580,0.000226,0.000729,0.000673,0.000053,0.000485
2024-09,0.001041,0.000117,0.000211,0.000827,0.000821,0.000665
2024-10,0.000818,0.000183,0.000288,0.000705,0.000336,0.000527
2024-11,0.000015,0.000007,0.000058,0.000235,0.000230,0.000157


In [120]:
dfresumo.index

Index(['2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07',
       '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01',
       '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07',
       '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01',
       '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07',
       '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01',
       '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07',
       '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01',
       '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07',
       '2024-08', '2024-09', '2024-10', '2024-11'],
      dtype='object', name='mes')

In [121]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='index',
                      values=dfregiaouf['obitosNovos'], aggfunc="sum",
                      margins=True, margins_name = 'Total')
print(dfresumo.loc['Total'].sum())
dfresumo.tail(5)

1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2024-08,0.112717,0.089595,0.109827,0.670520,0.017341
2024-09,0.147368,0.033684,0.023158,0.600000,0.195789
2024-10,0.146277,0.066489,0.039894,0.646277,0.101064
2024-11,0.008929,0.008929,0.026786,0.723214,0.232143
Total,0.094180,0.191796,0.072981,0.482492,0.158551
